In [1]:
from heapq import merge
from operator import index
from sys import displayhook
import mysql.connector
from mysql.connector import Error
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import mysql.connector as sql
import pandas as pd
from IPython.display import display

# 1. Pilih Database
connection = mysql.connector.connect(host='localhost',
                                         database='cargo_route', # masukan nama database di kiri
                                         user='root',
                                         password='')
cursor = connection.cursor(dictionary=True)

In [2]:
#2. masukkan querry 
#a. querry untuk table sample (sample rute dengan asal dan tujuan berupa int/kode)
samplequery = "SELECT * FROM sample;" 
masterquery = "SELECT * FROM city;"

In [3]:
#3. map sebelum di scale 
maps = pd.read_sql(samplequery,connection)
route = pd.read_sql(samplequery,connection)
display(maps)

C:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,sample_id,origin,destination,distance,price,rate,duration,airline
0,1,1,3,430,13000,3.5,2,
1,2,1,5,537,15500,4.0,3,
2,3,1,4,407,12500,4.0,2,
3,4,1,10,909,19000,4.0,7,
4,5,1,11,1259,20250,4.0,10,
5,6,1,9,917,19250,4.5,7,
6,7,1,14,1397,21000,4.5,11,
7,8,1,7,671,16500,4.0,4,
8,9,1,6,808,18250,4.5,6,
9,10,3,11,1025,19750,4.5,8,


In [10]:
from sklearn.preprocessing import MinMaxScaler

norm  = route[route.columns[3:7]]
scaler = MinMaxScaler(feature_range=(5, 10))
scaler.fit(norm)
normed=scaler.transform(norm)

route[route.columns[3:7]] = normed
route



,sample_id,origin,destination,distance,price,rate,duration,airline
0,1,Jakarta,Yogyakarta,5.610445,6.250000,6.25,5.454545,
1,2,Jakarta,Solo,5.988003,7.291667,7.50,5.909091,
2,3,Jakarta,Semarang,5.529287,6.041667,7.50,5.454545,
3,4,Jakarta,Palangkaraya,7.300635,8.750000,7.50,7.727273,
4,5,Jakarta,Balikpapan,8.535639,9.270833,7.50,9.090909,
5,6,Jakarta,Banjarmasin,7.328864,8.854167,8.75,7.727273,
6,7,Jakarta,Makassar,9.022583,9.583333,8.75,9.545455,
7,8,Jakarta,Malang,6.460833,7.708333,7.50,6.363636,
8,9,Jakarta,Surabaya,6.944248,8.437500,8.75,7.272727,
9,10,Yogyakarta,Balikpapan,7.709951,9.062500,8.75,8.181818,


In [11]:
city = pd.read_sql(masterquery,connection)
display(city)

C:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,city_id,city_name
0,1,Jakarta
1,2,Bandung
2,3,Yogyakarta
3,4,Semarang
4,5,Solo
5,6,Surabaya
6,7,Malang
7,8,Denpasar
8,9,Banjarmasin
9,10,Palangkaraya


In [12]:
#map dengan asal dan tujuan string bukan int (hanya untuk cek/tampilkan daftar rute)
def map_transformed(origin,destination):
    for kodeC in maps[maps.columns[1]].values: # ambil kode primary yang ada di kolom origin
        for k,v in city.values: # ambil PK dan nama kota yang ada di table kota
            if kodeC == k: # jika PK di kolom origin yang sedang di baca sama dengan salah satu Pk di table kota,
                origin.append(v) # tambahkan nama kotanya.
    route[route.columns[1]] = origin # ubah kolom origin di dataframe berdasarkan list penampung
# untuk destination kurang lebih sama
    for kodeC in maps[maps.columns[2]].values:
        for k,v in city.values:
            if kodeC == k:
                destination.append(v)
    route[route.columns[2]] = destination
    display(route)
origin = []  #list menampung transformasi primary code di kolom origin
destination = [] #list menampung transformasi primary code di kolom destination
map_transformed(origin,destination)

,sample_id,origin,destination,distance,price,rate,duration,airline
0,1,Jakarta,Yogyakarta,5.610445,6.250000,6.25,5.454545,
1,2,Jakarta,Solo,5.988003,7.291667,7.50,5.909091,
2,3,Jakarta,Semarang,5.529287,6.041667,7.50,5.454545,
3,4,Jakarta,Palangkaraya,7.300635,8.750000,7.50,7.727273,
4,5,Jakarta,Balikpapan,8.535639,9.270833,7.50,9.090909,
5,6,Jakarta,Banjarmasin,7.328864,8.854167,8.75,7.727273,
6,7,Jakarta,Makassar,9.022583,9.583333,8.75,9.545455,
7,8,Jakarta,Malang,6.460833,7.708333,7.50,6.363636,
8,9,Jakarta,Surabaya,6.944248,8.437500,8.75,7.272727,
9,10,Yogyakarta,Balikpapan,7.709951,9.062500,8.75,8.181818,


In [13]:
def graph_maker(graph):
    w1,w2,w3,w4 = (1,1,1,1)
    for row in maps.values:
        origin_db = row[1]
        destination_db = row[2]
        distance_db = row[3]
        price_db = row[4]
        rating_db = row[5]
        duration_db = row[6]
        
        W = (distance_db*1/w1) + (price_db*1/w2) + (-rating_db*1/w3) + (duration_db*1/w4)

        if origin_db in graph:
            graph[origin_db][destination_db] = W
        else:
            graph[origin_db]={}
            graph[origin_db][destination_db] = W
    return print(graph)
graph = {}
graph_maker(graph)

{1: {3: 13428.5, 5: 16036.0, 4: 12905.0, 10: 19912.0, 11: 21515.0, 9: 20169.5, 14: 22403.5, 7: 17171.0, 6: 19059.5}, 3: {11: 20778.5}, 4: {6: 10254.5}, 6: {2: 18776.5, 8: 11801.0, 14: 18792.0, 17: 23681.0, 9: 14482.5}, 9: {11: 12337.0}, 11: {12: 15516.5, 13: 13961.0, 14: 15514.0}, 13: {14: 13962.0}, 14: {18: 20462.0, 15: 13687.5, 16: 17985.5, 17: 21237.0}}


In [14]:
def graph_transformed(route_graph):
    w1,w2,w3,w4 = (1,1,1,1)
    for row in route.values:
        origin_db = row[1]
        destination_db = row[2]
        distance_db = row[3]
        distance_db = row[3]
        price_db = row[4]
        rating_db = row[5]
        duration_db = row[6]
        
        W = (distance_db*1/w1) + (price_db*1/w2) + (-rating_db*1/w3) + (duration_db*1/w4)

        if origin_db in route_graph:
            route_graph[origin_db][destination_db] = W
        else:
            route_graph[origin_db]={}
            route_graph[origin_db][destination_db] = W
    return print(route_graph)
transformed_graph = {}
graph_transformed(transformed_graph)


{'Jakarta': {'Yogyakarta': 11.064990055815745, 'Solo': 11.688760398622783, 'Semarang': 9.525499347747056, 'Palangkaraya': 16.27790787194457, 'Balikpapan': 19.397381097495774, 'Banjarmasin': 15.1603031906931, 'Makassar': 19.40137080045337, 'Malang': 13.032802442206112, 'Surabaya': 13.904475684865593}, 'Yogyakarta': {'Balikpapan': 16.204268781677037}, 'Semarang': {'Surabaya': 8.75}, 'Surabaya': {'Bandung': 13.23284895532602, 'Denpasar': 10.787314749470713, 'Makassar': 14.535777677979514, 'Manado': 20.0, 'Banjarmasin': 11.672200124034987}, 'Banjarmasin': {'Balikpapan': 11.122677017172428}, 'Balikpapan': {'Tarakan': 12.659855435084793, 'Palu': 10.139766578986762, 'Makassar': 11.402798272064327}, 'Palu': {'Makassar': 12.639766578986762}, 'Makassar': {'Ambon': 16.662670280789545, 'Kendari': 11.200913956074507, 'Gorontalo': 12.775677113406472, 'Manado': 16.2469258570176}}


In [15]:
def shortest(maps, asal, tujuan):
    result = []
    openlist = {}
    visitedlist = []
    labels = 'Jalur Tidak Di Temukan'
    
    result.append(asal)
   
    while tujuan not in result:
        current_node = result[-1]
        if current_node in maps :
            if current_node not in visitedlist:
                openlist = maps[current_node]
                print ('Open List :',openlist)
                visitedlist.append(current_node)
                jarak_terpendek = min(openlist.values())
                if tujuan not in (openlist):
                    for node, jarak in openlist.items():
                        if jarak == jarak_terpendek:
                            result.append(node)



                elif tujuan in (openlist):
                    for node in (maps[current_node]):
                        if node == tujuan:
                            result.append(node)



            elif current_node in visitedlist:
                result=labels
                print("rute tidak ditemukan")
                break



        elif current_node not in maps:
            result=labels
            print("rute tidak ditemukan")
            break


    return result
            

In [16]:
a = shortest(transformed_graph,'Jakarta','Gorontalo')
print ("\nJalur  :",a)

Open List : {'Yogyakarta': 11.064990055815745, 'Solo': 11.688760398622783, 'Semarang': 9.525499347747056, 'Palangkaraya': 16.27790787194457, 'Balikpapan': 19.397381097495774, 'Banjarmasin': 15.1603031906931, 'Makassar': 19.40137080045337, 'Malang': 13.032802442206112, 'Surabaya': 13.904475684865593}
Open List : {'Surabaya': 8.75}
Open List : {'Bandung': 13.23284895532602, 'Denpasar': 10.787314749470713, 'Makassar': 14.535777677979514, 'Manado': 20.0, 'Banjarmasin': 11.672200124034987}
rute tidak ditemukan

Jalur  : Jalur Tidak Di Temukan
